# GPU修复后验证

运行这个笔记本来验证GPU修复是否成功

## 🔍 驱动状态检查

In [ ]:
import subprocess
import sys

print("=== NVIDIA驱动状态 ===")

# 检查nvidia-smi
try:
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    if result.returncode == 0:
        print("✅ nvidia-smi 工作正常")
        print("输出前10行:")
        print('\n'.join(result.stdout.split('\n')[:10]))
    else:
        print("❌ nvidia-smi 失败:")
        print(result.stderr)
except Exception as e:
    print(f"❌ nvidia-smi 错误: {e}")

# 检查驱动版本
print("\n=== 驱动版本信息 ===")
try:
    with open('/proc/driver/nvidia/version', 'r') as f:
        version_info = f.read()
        print(version_info)
except:
    print("无法读取驱动版本信息")

## 🎯 PyTorch CUDA测试

In [ ]:
import torch
import torch.nn as nn

print("=== PyTorch CUDA测试 ===")
print(f"PyTorch版本: {torch.__version__}")
print(f"CUDA可用: {torch.cuda.is_available()}")
print(f"CUDA版本: {torch.version.cuda}")
print(f"cuDNN版本: {torch.backends.cudnn.version()}")

if torch.cuda.is_available():
    print(f"CUDA设备数量: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"设备 {i}: {torch.cuda.get_device_name(i)}")
        print(f"  总内存: {torch.cuda.get_device_properties(i).total_memory / 1024**3:.1f} GB")
else:
    print("⚠️ CUDA不可用 - 检查驱动安装")

In [ ]:
# GPU计算测试
if torch.cuda.is_available():
    print("=== GPU计算测试 ===")
    
    # 创建大矩阵
    size = 2000
    x = torch.randn(size, size, device='cuda')
    y = torch.randn(size, size, device='cuda')
    
    print(f"创建矩阵: {size}x{size}")
    print(f"内存使用: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    
    # 矩阵乘法
    import time
    start = time.time()
    z = torch.matmul(x, y)
    torch.cuda.synchronize()  # 等待GPU完成
    end = time.time()
    
    print(f"矩阵乘法完成")
    print(f"计算时间: {end - start:.3f} 秒")
    print(f"结果形状: {z.shape}")
    print(f"最终内存使用: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    
    # 清理
    del x, y, z
    torch.cuda.empty_cache()
    
else:
    print("跳过GPU测试 - CUDA不可用")

## 🔥 神经网络训练测试

In [ ]:
if torch.cuda.is_available():
    print("=== 简单神经网络训练测试 ===")
    
    # 创建简单的网络
    class SimpleNet(nn.Module):
        def __init__(self, input_size, hidden_size, output_size):
            super().__init__()
            self.fc1 = nn.Linear(input_size, hidden_size)
            self.relu = nn.ReLU()
            self.fc2 = nn.Linear(hidden_size, output_size)
        
        def forward(self, x):
            x = self.fc1(x)
            x = self.relu(x)
            x = self.fc2(x)
            return x
    
    # 网络参数
    input_size = 784
    hidden_size = 128
    output_size = 10
    
    # 创建网络和优化器
    net = SimpleNet(input_size, hidden_size, output_size).cuda()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters(), lr=0.001)
    
    print(f"网络结构: {input_size} -> {hidden_size} -> {output_size}")
    print(f"总参数: {sum(p.numel() for p in net.parameters()):,}")
    
    # 生成随机数据
    batch_size = 64
    X = torch.randn(batch_size, input_size).cuda()
    y = torch.randint(0, output_size, (batch_size,)).cuda()
    
    # 训练几个epoch
    net.train()
    for epoch in range(3):
        optimizer.zero_grad()
        outputs = net(X)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()
        
        if epoch == 0:
            print(f"第1个epoch损失: {loss.item():.4f}")
    
    print(f"最终损失: {loss.item():.4f}")
    print("✅ GPU训练测试完成!")
    
    # 清理
    del net, X, y
    torch.cuda.empty_cache()
    
else:
    print("跳过神经网络测试 - CUDA不可用")

## 🎯 项目集成测试

In [ ]:
# 测试项目中的功能
import sys
import os
sys.path.append(os.path.join(os.getcwd(), '../src'))

print("=== 项目集成测试 ===")

try:
    from utils import set_seed, get_device, print_gpu_memory
    from trainer import create_sample_data
    
    # 设置随机种子
    set_seed(42)
    
    # 获取设备
    device = get_device()
    print(f"项目检测设备: {device}")
    
    # 打印GPU内存
    print_gpu_memory()
    
    # 创建样本数据
    sample_texts = create_sample_data()
    print(f"✅ 创建样本数据: {len(sample_texts)} 条文本")
    
    if torch.cuda.is_available():
        print("🎉 GPU修复成功！可以加速训练了！")
    else:
        print("⚠️ GPU仍然不可用，但可以继续CPU学习")
    
except Exception as e:
    print(f"❌ 项目测试失败: {e}")
    print("检查项目文件完整性")

## 📋 下一步行动清单

根据测试结果选择下一步：

### 如果GPU修复成功 ✅
1. 运行 `python main.py --test generation` 测试文本生成
2. 开始 `notebooks/02_simple_llm_demo.ipynb` 学习
3. 尝试训练更复杂的模型

### 如果GPU仍然有问题 ❌
1. 检查驱动安装日志
2. 尝试不同的驱动版本（535/570/470）
3. 继续CPU模式学习（完全可行）
4. 寻求社区帮助

## 🔗 有用命令

```bash
# 检查驱动状态
nvidia-smi
cat /proc/driver/nvidia/version
lsmod | grep nvidia

# 重新安装驱动
sudo ./fix_gpu_quick.sh

# 测试PyTorch
python -c "import torch; print(torch.cuda.is_available())"
```